In [124]:
import os
import utils
import numpy as np
import pandas as pd

In [125]:
def read_all_results(dir_path):
    paths = os.listdir(dir_path)
    last_gens = []
    for i in range(len(paths)):
        path = dir_path + paths[i]
        last_gen = sorted(map(lambda x: x.split("_"), 
                        os.listdir(f"{path}/individuals/")), 
                        key=lambda x: int(x[1].split(".")[0]))[-1][-1][:-4]
        print(f"Path: {paths[i]}, last generation: {int(last_gen)}")
        last_gens.append(int(last_gen))
    return paths, last_gens

In [126]:
dir_path = "results/"
paths, last_gens = read_all_results(dir_path)

Path: GA_2021_05_30_15_48_02, last generation: 6


In [127]:
run = 0
gen = 6
path = dir_path + paths[run]

individuals_path =  f"{path}/individuals/individuals_{gen}.pkl"
score_path       =  f"{path}/final_scores/final_score_{gen}.pkl"
overview_path    =  f"{path}/generation_overview.csv"

final_score = utils.read_pickle(score_path)
individuals = utils.read_pickle(individuals_path)
overview = pd.read_csv(overview_path)

In [128]:
with open(dir_path + paths[run] + "/run_params.json", "r") as file:
    print(file.read())
    file.close()

{
    "objective": "deaths",
    "simulations": 3,
    "process": {
        "horizon": 74,
        "decision_period": 28,
        "policy": "weighted"
    },
    "population_size": 10,
    "random_individuals": false,
    "min_generations": 10
}


In [129]:
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
print("Strategy_count:")
print(f"{utils.tcolors.BOLD}{'#':<3} {'U':<5}{'D':<5}{'N':<5}{utils.tcolors.ENDC}")
for trend_count in range(3):
    strategy_count = individuals[0].strategy_count[gen]
    print(f"{trend_count+1:<2}| {strategy_count['U'][trend_count]:<5}{strategy_count['D'][trend_count]:<5}{strategy_count['N'][trend_count]:<5}")
print()
for ix, i in enumerate(sorted(individuals, key=lambda x: x.mean_score)):
    print(f"{utils.tcolors.BOLD}{ordinal(ix+1)} place:{utils.tcolors.ENDC}")
    print(f"ID: {i.ID}")
    print(f"Score: {np.round(i.mean_score, 3)}")
    print(f"Genes:\n{np.round(i.genes, 3)}")
    print()

Strategy_count:
#   U    D    N    
1 | 3    3    3    
2 | 2    1    3    
3 | 0    0    3    

1st place:
ID: gen_5_011
Score: 866.979
Genes:
[[[0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]]]

2nd place:
ID: gen_6_001
Score: 893.698
Genes:
[[[0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]]]

3rd place:
ID: gen_4_014
Score: 931.747
Genes:
[[[0.028 0.    0.    0.972 0.   ]
  [0.    0.    0.125 0.875 0.   ]
  [0.    0.    0.144 0.856 0.   ]]

 [[0.111 0.    0.111 0.778 0.   ]
  [0.    0.    0.125 0.875 0.   ]
  [0.    0.    0.094 0.906 0.   ]]

 [[0.    0.    0.125 0.875 0.   ]
  [0.    0.    0.    1.    0.   ]
  [0.    0.122 0.122 0.756 0.   ]]]

4th place:
ID: gen_3_011
Score: 943.132
Genes:
[[[0.111 0.    0.    0.889 0.   ]
  [0.    0. 

In [123]:
print(individuals[0].genes, end="\n"*2)
genes = np.sum(individuals[0].genes, axis=(0,1))
norm = np.sum(genes)
norm_genes = np.divide(genes, norm)
print(norm_genes)

[[[0.         0.         0.8125     0.1875     0.        ]
  [0.         0.         1.         0.         0.        ]
  [0.         0.         0.8125     0.1875     0.        ]]

 [[0.         0.         0.8125     0.1875     0.        ]
  [0.         0.         0.8125     0.1875     0.        ]
  [0.15789474 0.         0.68421053 0.15789474 0.        ]]

 [[0.         0.         0.8125     0.1875     0.        ]
  [0.         0.         0.8125     0.1875     0.        ]
  [0.         0.         0.8125     0.1875     0.        ]]]

[0.01754386 0.         0.81907895 0.16337719 0.        ]
